<a href="https://colab.research.google.com/github/damianwgriggs/The-Griggs-Cold-Chain/blob/main/griggschainexperiment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# ==========================================
# MASTER CELL: PRE-FREEZED FREEZER PROTOCOL (DD + RECURSIVE)
# ==========================================

!pip install -q qiskit qiskit-ibm-runtime qiskit-aer numpy

import numpy as np
import socket
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# --- PART 1: THE "PRE-FREEZED" ARCHITECTURE ---
def create_prefreezed_circuit(travel_time_us=10, hold_time_us=20):
    """
    Implements Damian Griggs' "Prefreezed Freezer" Logic.
    1. Pre-Cooling: Echo on q2 while q0 prepares (The Freezer is On).
    2. Delivery: Teleportation.
    3. Post-Cooling: Echo on q2 after delivery (Keeping it Frozen).
    """
    qr = QuantumRegister(3, 'q')
    cr_hop1 = ClassicalRegister(2, 'hop1')
    cr_hop2 = ClassicalRegister(2, 'hop2')
    cr_final = ClassicalRegister(1, 'result')
    qc = QuantumCircuit(qr, cr_hop1, cr_hop2, cr_final)

    # --- PHASE 0: THE "PRE-FREEZE" (Simultaneous Ops) ---
    # While q0 prepares (The "Truck"), q2 keeps pulsing (The "Freezer")

    # Lane 1: The Truck (Source q0)
    qc.ry(2 * np.arccos(np.sqrt(0.75)), 0) # Load the cargo (75% State)
    if travel_time_us > 0:
        qc.delay(travel_time_us, 0, unit='us') # The Drive

    # Lane 2: The Freezer (Target q2) - Pre-Cooling
    # We apply an X-Echo during the exact time q0 is busy
    if travel_time_us > 0:
        qc.delay(travel_time_us / 2, 2, unit='us')
        qc.x(2) # PRE-ECHO PULSE (Cleaning the empty freezer)
        qc.delay(travel_time_us / 2, 2, unit='us')
        qc.x(2) # Restore orientation for reception

    qc.barrier()

    # --- PHASE 1: THE DELIVERY (Teleportation) ---
    qc.h(1)
    qc.cx(1, 2) # Entangle Bridge & Buffer
    qc.cx(0, 1) # Entangle Source & Bridge
    qc.h(0)
    qc.measure(0, cr_hop1[0])
    qc.measure(1, cr_hop1[1])

    # Dynamic Correction (Unloading)
    with qc.if_test((cr_hop1[1], 1)):
        qc.x(2)
    with qc.if_test((cr_hop1[0], 1)):
        qc.z(2)
    qc.barrier()

    # --- PHASE 2: RECYCLING & HOLDING (Post-Freeze) ---
    # 1. Send the truck back (Recycle q0)
    qc.reset(0)
    qc.reset(1)

    # 2. Keep the Freezer Running (Stabilize q2)
    if hold_time_us > 0:
        qc.delay(hold_time_us / 2, 2, unit='us')
        qc.x(2) # POST-ECHO PULSE
        qc.delay(hold_time_us / 2, 2, unit='us')
        qc.x(2)

    qc.barrier()

    # --- PHASE 3: VERIFICATION ---
    # Return to sender to check if it melted
    qc.ry(-2 * np.arccos(np.sqrt(0.75)), 2) # Inversion check on q2 directly?
    # Or teleport back? Let's use your original "Teleport Back" to prove full routing loop.

    # Teleport back to q0 (The full round trip)
    qc.h(1)
    qc.cx(1, 0)
    qc.cx(2, 1)
    qc.h(2)
    qc.measure(2, cr_hop2[0])
    qc.measure(1, cr_hop2[1])

    with qc.if_test((cr_hop2[1], 1)):
        qc.x(0)
    with qc.if_test((cr_hop2[0], 1)):
        qc.z(0)

    # Final Check
    qc.measure(0, cr_final)

    return qc

# --- PART 2: DEPLOYMENT ---
MY_TOKEN = "MYTOKENWASHERE"

if MY_TOKEN == "YOUR_TOKEN_HERE" or MY_TOKEN == "":
    print("STOP: Please paste your IBM Token.")
else:
    try:
        service = QiskitRuntimeService(channel="ibm_quantum_platform", token=MY_TOKEN)

        # Hunting for the best available machine
        print("[Step 1] Initializing Cold Chain Protocol...")
        backend = service.least_busy(min_num_qubits=127, operational=True, simulator=False)
        print(f"         Targeting: {backend.name}")

        # Generate Circuit with 10us Pre-Cool and 20us Post-Hold
        print("[Step 2] Activating Prefreezed Freezer (10us Pre / 20us Post)...")
        qc_cold = create_prefreezed_circuit(travel_time_us=10, hold_time_us=20)

        # Optimization Level 3 is vital for scheduling the parallel delays correctly
        print("         Compiling with Dynamical Decoupling...")
        pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
        isa_qc = pm.run(qc_cold)

        print("[Step 3] Executing...")
        sampler = Sampler(backend)
        job = sampler.run([isa_qc], shots=8192)
        print(f"         Job ID: {job.job_id()}")

        result = job.result()
        counts = result[0].data.result.get_counts()

        zeros = 0
        total = 0
        for k, v in counts.items():
            if k.startswith('0'):
                zeros += v
            total += v

        fidelity = (zeros/total)*100

        print("\n" + "*"*45)
        print(f" DAMIAN GRIGGS: COLD CHAIN REPORT")
        print("*"*45)
        print(f"Architecture:   Pre-Freezed Recursive Link")
        print(f"Pre-Cooling:    ACTIVE (10us)")
        print(f"Post-Holding:   ACTIVE (20us)")
        print("-" * 45)
        print(f"FINAL FIDELITY: {fidelity:.2f}%")
        print("*"*45)

        if fidelity > 65.05:
            print("❄️ VALIDATED: Pre-cooling improved the stability.")

    except Exception as e:
        print(f"Error: {e}")

qiskit_runtime_service._discover_account:WARNING:2025-12-19 15:28:31,841: Loading account with the given token. A saved account will not be used.
qiskit_runtime_service.__init__:WARNING:2025-12-19 15:28:34,110: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().


[Step 1] Initializing Cold Chain Protocol...


qiskit_runtime_service.backends:WARNING:2025-12-19 15:28:34,615: Loading instance: open-instance, plan: open
qiskit_runtime_service.backends:WARNING:2025-12-19 15:28:36,230: Using instance: open-instance, plan: open


         Targeting: ibm_torino
[Step 2] Activating Prefreezed Freezer (10us Pre / 20us Post)...
         Compiling with Dynamical Decoupling...
[Step 3] Executing...
         Job ID: d52mu9fp3tbc73alnd2g

*********************************************
 DAMIAN GRIGGS: COLD CHAIN REPORT
*********************************************
Architecture:   Pre-Freezed Recursive Link
Pre-Cooling:    ACTIVE (10us)
Post-Holding:   ACTIVE (20us)
---------------------------------------------
FINAL FIDELITY: 67.37%
*********************************************
❄️ VALIDATED: Pre-cooling improved the stability.
